In [1]:
import pandas as pd
df = pd.read_csv("atp_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44708 entries, 0 to 44707
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ATP         44708 non-null  int64  
 1   Location    44708 non-null  object 
 2   Tournament  44708 non-null  object 
 3   Date        44708 non-null  object 
 4   Series      44708 non-null  object 
 5   Court       44708 non-null  object 
 6   Surface     44708 non-null  object 
 7   Round       44708 non-null  object 
 8   Best of     44708 non-null  int64  
 9   Winner      44708 non-null  object 
 10  Loser       44708 non-null  object 
 11  WRank       44708 non-null  int64  
 12  LRank       44708 non-null  int64  
 13  Wsets       44521 non-null  float64
 14  Lsets       44521 non-null  float64
 15  Comment     44708 non-null  object 
 16  PSW         32743 non-null  float64
 17  PSL         32743 non-null  float64
 18  B365W       39037 non-null  float64
 19  B365L       39057 non-nul

# Transformations

In [2]:
#Suppression des colonnes non pertinentes
df = df.drop(["Wsets","Lsets", "Comment", "Location","Date"], axis = 1)
# Remplacer les valeurs manquantes par la moyenne de chaque colonne numérique
df.fillna(df.mean(), inplace=True)
df.head()

/tmp/ipykernel_18769/1570066895.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


,ATP,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,PSW,PSL,B365W,B365L,elo_winner,elo_loser,proba_elo
0,1,Australian Hardcourt Championships,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63,77,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5
1,3,Qatar Open,International,Outdoor,Hard,1st Round,3,Kiefer N.,Tarango J.,6,59,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5
2,3,Qatar Open,International,Outdoor,Hard,1st Round,3,Gaudio G.,Luxa P.,73,174,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5
3,3,Qatar Open,International,Outdoor,Hard,1st Round,3,El Aynaoui Y.,Dupuis A.,33,78,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5
4,3,Qatar Open,International,Outdoor,Hard,1st Round,3,Cherkasov A.,Arazi H.,206,35,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5


In [3]:
# Création de 2 nouvelles colonnes Joueur 1 et joueur 2 + isolation target
import random

df_transformed = df

df_transformed['joueur1'] = ''
df_transformed['joueur2'] = ''
df_transformed['target'] = ''

for index, row in df_transformed.iterrows():
    if random.choice([True, False]):
        df_transformed.at[index, 'joueur1'] = row['Winner']
        df_transformed.at[index, 'joueur2'] = row['Loser']
        df_transformed.at[index, 'target'] = 0
    else:
        df_transformed.at[index, 'joueur1'] = row['Loser']
        df_transformed.at[index, 'joueur2'] = row['Winner']
        df_transformed.at[index, 'target'] = 1

df_transformed['target'] = df_transformed.apply(lambda row: 0 if row['Winner'] == row['joueur1'] else 1, axis=1)


target = df_transformed["target"]
df_transformed = df_transformed.drop(['Winner', 'Loser', "target"], axis=1)

# Labelliser ? Encodage des noms des joueurs + Rank ?

In [4]:
names_combination = []

for index, row in df_transformed.iterrows():
    v = []
    v.append(row['joueur1'])
    v.append(row['joueur2'])
    names_combination.append(v)
    

    
# Confirmer ou pas le maintien de joueur 1 / joueur 2 en dychotomie Ou préférable de labelliser avant 
# Créez une instance de LabelEncoder pour chaque colonne
le_colonne1 = LabelEncoder()
le_colonne2 = LabelEncoder()


# Ajustez chaque LabelEncoder aux valeurs uniques des colonnes respectives
le_colonne2.fit(df_transformed['joueur1'])
le_colonne3.fit(df_transformed['joueur2'])

# Transformez les colonnes en valeurs numériques
df_transformed['joueur1_Encoded'] = le_colonne2.transform(df_transformed['joueur1'])
df_transformed['joueur2_Encoded'] = le_colonne3.transform(df_transformed['joueur2'])

In [5]:
columns_to_dichotomize = ["Tournament", "Series","Court", "Surface", "Round", "Best of", "joueur1", "joueur2"]

def dychotomize(df_fonc, columns_to_dichotomize):
    for element in columns_to_dichotomize:
        df_fonc = df_fonc.join(pd.get_dummies(df_fonc[element], prefix=element), how='left')
        df_fonc = df_fonc.drop(element,1)
    return df_fonc

df_transformed = dychotomize(df_transformed, columns_to_dichotomize)
df_transformed.head()

/tmp/ipykernel_18769/3254653772.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_fonc = df_fonc.drop(element,1)
/tmp/ipykernel_18769/3254653772.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_fonc = df_fonc.drop(element,1)
/tmp/ipykernel_18769/3254653772.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_fonc = df_fonc.drop(element,1)
/tmp/ipykernel_18769/3254653772.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_fonc = df_fonc.drop(element,1)
/tmp/ipykernel_18769/3254653772.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_f

,ATP,WRank,LRank,PSW,PSL,B365W,B365L,elo_winner,elo_loser,proba_elo,...,joueur2_Zverev A.,joueur2_Zverev M.,joueur2_de Chaunac S.,joueur2_de Voest R.,joueur2_di Mauro A.,joueur2_di Pasquale A.,joueur2_van Gemerden M.,joueur2_van Lottum J.,joueur2_van Scheppingen D.,joueur2_van der Meer N.
0,1,63,77,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5,...,0,0,0,0,0,0,0,0,0,0
1,3,6,59,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5,...,0,0,0,0,0,0,0,0,0,0
2,3,73,174,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5,...,0,0,0,0,0,0,0,0,0,0
3,3,33,78,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5,...,0,0,0,0,0,0,0,0,0,0
4,3,206,35,1.927563,4.240179,1.822246,3.551007,1500.0,1500.0,0.5,...,0,0,0,0,0,0,0,0,0,0


# Preprocessing

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(df_transformed, target, test_size=0.2, random_state=66)

scaler = MinMaxScaler()
df_transformed[df_transformed.columns] = scaler.fit_transform(df_transformed[df_transformed.columns])



# Regression logistique

In [23]:
from sklearn import linear_model

clf = linear_model.LogisticRegression(C=1.0)
clf.fit(X_train, y_train)


/home/rui/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
y_pred = clf.predict(X_test)
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(cm)

Classe prédite     0     1
Classe réelle             
0               2802  1728
1               2195  2217


In [27]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.56      0.62      0.59      4530
           1       0.56      0.50      0.53      4412

    accuracy                           0.56      8942
   macro avg       0.56      0.56      0.56      8942
weighted avg       0.56      0.56      0.56      8942



# SVM

In [ ]:
from sklearn import svm

clf2 = svm.SVC(gamma=0.01,  kernel='poly')
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

In [ ]:
cm2 = pd.crosstab(y_test, y_pred2, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(cm2)

In [ ]:
report2 = classification_report(y_test, y_pred2)
print(report2)

# KNN

In [7]:
from sklearn import neighbors

knn = neighbors.KNeighborsClassifier(n_neighbors=7, metric='minkowski')
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

In [8]:
knnres = pd.crosstab(y_test, y_pred_knn, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(knnres)

Classe prédite     0     1
Classe réelle             
0               2219  2301
1               2214  2208


In [10]:
from sklearn.metrics import classification_report
report3 = classification_report(y_test, y_pred_knn)
print(report3)

              precision    recall  f1-score   support

           0       0.50      0.49      0.50      4520
           1       0.49      0.50      0.49      4422

    accuracy                           0.50      8942
   macro avg       0.50      0.50      0.50      8942
weighted avg       0.50      0.50      0.50      8942



# Arbre de décision

In [11]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(criterion ='entropy', max_depth=4, random_state=123)
dt_clf.fit(X_train, y_train)
y_pred_tree = dt_clf.predict(X_test)

In [12]:
treeres = pd.crosstab(y_test, y_pred_tree, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(treeres)

Classe prédite    0     1
Classe réelle            
0               245  4275
1                22  4400


In [13]:
report4 = classification_report(y_test, y_pred_tree)
print(report4)

              precision    recall  f1-score   support

           0       0.92      0.05      0.10      4520
           1       0.51      1.00      0.67      4422

    accuracy                           0.52      8942
   macro avg       0.71      0.52      0.39      8942
weighted avg       0.71      0.52      0.38      8942



# Forets Aleatoires

In [14]:
from sklearn import ensemble

forest_clf = ensemble.RandomForestClassifier(n_jobs=-1, random_state=321)
forest_clf.fit(X_train, y_train)
y_pred_forest = forest_clf.predict(X_test)

In [15]:
forestres = pd.crosstab(y_test, y_pred_forest, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(forestres)

Classe prédite     0     1
Classe réelle             
0               3500  1020
1                917  3505


In [16]:
report5 = classification_report(y_test, y_pred_forest)
print(report5)

              precision    recall  f1-score   support

           0       0.79      0.77      0.78      4520
           1       0.77      0.79      0.78      4422

    accuracy                           0.78      8942
   macro avg       0.78      0.78      0.78      8942
weighted avg       0.78      0.78      0.78      8942



# Boosting Bagging ? XGBoost ?

# Utilisation pour predire ulterieurement

In [ ]:
#Exemple de code

new_data = {
    'player1': player1,
    'player2': player2,

    'Elo_Player1': elo_player1, #get d'une api de classement pour avoir classement en temps reel?
    'Elo_Player2': elo_player2
}

# Transformez le dictionnaire en un DataFrame
new_data_df = pd.DataFrame([new_data])

#Encoder les nom des joueurs si besoin

# Utilisez le modèle pour effectuer la prédiction.
result = model.predict(new_data_df)

# Le résultat sera la classe prédite 
print(f"Résultat de la prédiction : {result[0]}")